In [1]:
#Install required libraries
!pip install fastapi uvicorn psycopg2-binary sqlalchemy python-dotenv groq


In [2]:
import os
from google.colab import userdata

GROQ_API_KEY = userdata.get("GROQ_API_KEY")
DATABASE_URL = userdata.get("DATABASE_URL")

assert GROQ_API_KEY is not None, "Missing GROQ_API_KEY"
assert DATABASE_URL is not None, "Missing DATABASE_URL"

In [3]:
# Database Connection

import psycopg2
import os

def get_connection():
    return psycopg2.connect(DATABASE_URL)

In [4]:
#SQL Safety Guard

def is_safe_sql(sql: str) -> bool:
    forbidden = ["insert", "update", "delete", "drop", "truncate", "alter"]
    sql_lower = sql.lower()
    return (
        sql_lower.strip().startswith("select")
        and not any(word in sql_lower for word in forbidden)
    )




In [5]:
#row limit safety:

def enforce_limit(sql, limit=1000):
    if "limit" not in sql.lower():
        return sql.rstrip(";") + f" LIMIT {limit};"
    return sql

In [6]:
# SQL Executor

from psycopg2.extras import RealDictCursor

def execute_sql(conn, sql):
    with conn.cursor(cursor_factory=RealDictCursor) as cur:
        cur.execute(sql)
        return cur.fetchall()

In [7]:
#NL2SQL System Prompt

# system_prompt
SYSTEM_PROMPT = """
You are an expert PostgreSQL NL2SQL agent.

Your task is to convert a user’s natural language business question
(English or Hinglish) into a correct, safe PostgreSQL SELECT query.

If the question is in Hindi/Hinglish, internally translate it to English
before generating SQL.

Return ONLY the SQL query.
Do NOT add explanations, comments, or markdown.

--------------------------------------------------
DATABASE SCHEMA
--------------------------------------------------

table_first:
- date, firm, firm_product, voucher_type, voucher_no,
  gstin_uin, quantity, rate, value, brand,
  sub_category, category, client_type

table_second:
- date, firm, voucher_no, quantity, value,
  gstin_uin, total_tax_amount, gross_total

--------------------------------------------------
BUSINESS SEMANTIC LAYER
--------------------------------------------------

- Sales / business / turnover / revenue → gross_total (table_second)
- Net sales → value
- GST / tax → total_tax_amount
- Product / brand / category / quantity → table_first

--------------------------------------------------
JOIN RULES
--------------------------------------------------

- Do NOT use JOIN unless strictly required
- JOIN ONLY on voucher_no
- If a query can be answered using table_second alone,
  JOIN is INCORRECT
- Avoid joins that cause double counting

--------------------------------------------------
AGGREGATION RULES
--------------------------------------------------

- Always use COALESCE(SUM(column), 0)
- Do not aggregate unless explicitly asked

--------------------------------------------------
GROUPING (“WISE”) RULES
--------------------------------------------------

If question contains "firm wise", "brand wise", "category wise",
then:
- SELECT that column
- GROUP BY that column

--------------------------------------------------
DATE RULES
--------------------------------------------------

- Use EXTRACT(YEAR FROM date) and EXTRACT(MONTH FROM date)
- Month filters MUST include year

Current:
- "is month" → CURRENT_DATE month + year
- "is saal" → CURRENT_DATE year

Past:
- "pichle saal" → current_year - 1
- "pichle N saal" → BETWEEN current_year-N AND current_year-1

Future:
- Only filter existing data
- Do NOT predict

--------------------------------------------------
LIMIT & ORDER
--------------------------------------------------

- Add LIMIT only if user asks
- Top / highest → ORDER BY aggregate DESC
- Lowest → ORDER BY aggregate ASC

--------------------------------------------------
OUTPUT
--------------------------------------------------

Return ONLY valid PostgreSQL SELECT SQL.
"""



In [14]:
#NL2SQL Agent
# nl2sql_agent
from groq import Groq
import os

client = Groq(api_key=GROQ_API_KEY)

def nl_to_sql(question: str) -> str:
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": question}
        ],
        temperature=0
    )
    return response.choices[0].message.content.strip()

In [9]:
# FastAPI Endpoint
# app
from fastapi import FastAPI, HTTPException

app = FastAPI(title="NL2SQL Analytics API")

@app.post("/query")
def query_db(question: str):
    sql = nl_to_sql(question)

    if not is_safe_sql(sql):
        raise HTTPException(status_code=400, detail="Unsafe SQL blocked")

    sql = enforce_limit(sql)

    conn = get_connection()
    data = execute_sql(conn, sql)
    conn.close()

    return {
        "question": question,
        "sql": sql,
        "data": data
    }

In [10]:
import nest_asyncio
import uvicorn
import threading
import time

nest_asyncio.apply()

def run_uvicorn_in_thread():
    # Use a lower log_level to reduce console output in Colab
    uvicorn.run(app, host="0.0.0.0", port=8001, log_level="warning")

# Create and start the thread
thread = threading.Thread(target=run_uvicorn_in_thread)
thread.start()

# Give Uvicorn a moment to start up
time.sleep(2)
print("FastAPI app is running in the background on http://0.0.0.0:8001")

FastAPI app is running in the background on http://0.0.0.0:8001


##Testing the Endpoint


In [11]:
import requests

# Define the base URL for your FastAPI application
BASE_URL = "http://localhost:8001" # Updated port to 8001



In [67]:
# Example 1
question1 = "What was firmwise sales in December 2025 "
response1 = requests.post(f"{BASE_URL}/query", params={"question": question1})

print(f"Question 1: {question1}")
print(f"Status Code: {response1.status_code}")
print(f"Response : {response1.json()}")




Question 1: What was firmwise sales in December 2025 
Status Code: 200
Response : {'question': 'What was firmwise sales in December 2025 ', 'sql': 'SELECT EXTRACT(YEAR FROM date) AS year, EXTRACT(MONTH FROM date) AS month, firm, COALESCE(SUM(value), 0) AS sales \nFROM table_first \nWHERE EXTRACT(MONTH FROM date) = 12 AND EXTRACT(YEAR FROM date) = 2025 \nGROUP BY firm, EXTRACT(YEAR FROM date), EXTRACT(MONTH FROM date) LIMIT 1000;', 'data': [{'year': 2025, 'month': 12, 'firm': 'A TO Z SECUTECH PRIVATE LIMITED', 'sales': 1347652.0}, {'year': 2025, 'month': 12, 'firm': 'ABN CONNECT', 'sales': 44160.93}, {'year': 2025, 'month': 12, 'firm': 'ACS DIGITAL INDIA (AKASH)', 'sales': 104877.06}, {'year': 2025, 'month': 12, 'firm': 'ADINATH SECURITY SOLUTIONS', 'sales': 10000.0}, {'year': 2025, 'month': 12, 'firm': 'ADVANCE COMMUNICATION SERVICE', 'sales': 720.34}, {'year': 2025, 'month': 12, 'firm': 'ADVANCE SECURITY & IT', 'sales': 285593.41}, {'year': 2025, 'month': 12, 'firm': 'AJEEM', 'sales':

In [12]:
# Example 2:
question2 = "Category wise sales for December 2025"
response2 = requests.post(f"{BASE_URL}/query", params={"question": question2})

print(f"Question 2: {question2}")
print(f"Status Code: {response2.status_code}")
print(f"Response: {response2.json()}")

Question 2: Category wise sales for December 2025
Status Code: 200
Response: {'question': 'Category wise sales for December 2025', 'sql': 'SELECT EXTRACT(YEAR FROM date) AS year, EXTRACT(MONTH FROM date) AS month, category, COALESCE(SUM(value), 0) AS sales \nFROM table_first \nWHERE EXTRACT(MONTH FROM date) = 12 AND EXTRACT(YEAR FROM date) = 2025 \nGROUP BY category, EXTRACT(YEAR FROM date), EXTRACT(MONTH FROM date) LIMIT 1000;', 'data': [{'year': 2025, 'month': 12, 'category': 'Accessories', 'sales': 15810336.45}, {'year': 2025, 'month': 12, 'category': 'Cameras', 'sales': 8772320.47}]}


In [13]:
# Example 3:
question3 = "Give sales for November 2025"
response3 = requests.post(f"{BASE_URL}/query", params={"question": question2})

print(f"Question 3: {question3}")
print(f"Status Code: {response3.status_code}")
print(f"Response : {response3.json()}")

Question 3: Give sales for November 2025
Status Code: 200
Response : {'question': 'Category wise sales for December 2025', 'sql': 'SELECT EXTRACT(YEAR FROM date) AS year, EXTRACT(MONTH FROM date) AS month, category, COALESCE(SUM(value), 0) AS sales \nFROM table_first \nWHERE EXTRACT(MONTH FROM date) = 12 AND EXTRACT(YEAR FROM date) = 2025 \nGROUP BY category, EXTRACT(YEAR FROM date), EXTRACT(MONTH FROM date) LIMIT 1000;', 'data': [{'year': 2025, 'month': 12, 'category': 'Accessories', 'sales': 15810336.45}, {'year': 2025, 'month': 12, 'category': 'Cameras', 'sales': 8772320.47}]}
